The first step is to import the libraries and set the OpenAI API key and endpoint. You'll need to set the following environment variables:

- `AZURE_OPENAI_API_KEY` - Your OpenAI API key
- `AZURE_OPENAI_ENDPOINT` - Your OpenAI endpoint
- `AZURE_COG_SEARCH_API_KEY` - Your Azure Cognitive Search API key
- `AZURE_COG_SEARCH_ENDPOINT` - Your Azure Cognitive Search endpoint

In [ ]:
import os
import httpx
import pandas as pd
import openai
from openai.embeddings_utils import get_embedding


AZURE_COG_SEARCH_API_KEY = os.environ["AZURE_COG_SEARCH_API_KEY"]
AZURE_COG_SEARCH_SERVICE_NAME = os.environ["AZURE_COG_SEARCH_SERVICE_NAME"]
AZURE_COG_SEARCH_API_VERSION = "2023-07-01-Preview"
AZURE_COG_SEARCH_INDEX_NAME = "product-catalog-index"


openai.api_type = "azure"
openai.api_key = os.environ["AZURE_OPENAI_API_KEY"]
openai.api_base = os.environ["AZURE_OPENAI_ENDPOINT"]
openai.api_version = "2023-07-01-preview"

OPENAI_EMBEDDING_ENGINE = "text-embedding-ada-002"

OPENAI_EMBEDDING_DEPLOYMENT_NAME = os.environ[
    "AZURE_OPENAI_EMBEDDING_MODEL_DEPLOYMENT_NAME"
]

Next, we are going to create a function called `search` that will search the Embedding Index for the query. The function works as follows:


1. First, the Embedding for the query is calculated using the OpenAI Embedding API.
2. Post the data to the Azure OpenAI Service to search the Embedding Index.
3. Return the results.

In [ ]:
def search(query: str, rows: int):
    # get the embeddings for the query
    query_embeddings = get_embedding(query, OPENAI_EMBEDDING_ENGINE)

    request = {
        "vector": {
            "value": query_embeddings,
            "fields": "contentVector",
            "k": 5,
        },
        "select": "title, content, category",
    }

    url = (
        f"https://{AZURE_COG_SEARCH_SERVICE_NAME}.search.windows.net/indexes/{AZURE_COG_SEARCH_INDEX_NAME}"
        f"/docs/search?api-version={AZURE_COG_SEARCH_API_VERSION}"
    )

    headers = {
        "Content-Type": "application/json",
        "api-key": AZURE_COG_SEARCH_API_KEY,
    }

    response = httpx.post(url, json=request, headers=headers)
    response.raise_for_status()

    # get the results
    results = response.json()["value"]

    # return the top rows
    return results

This function is very simple, it just prints out the results of the search query.

In [ ]:
def display_results(results: list, query: str):
    print(f"\nQuery: '{query}':\n")
    for result in results:
        print(f" - {result['title']}")
        print(f"   search.score: {result['@search.score']}")
        print(f"   content: {result['content']}")


1. First, the user is prompted to enter a query.
2. Then the `search` function is called to search the Embedding Index for the query.
3. Finally, the `display_results` function is called to display the results to the user.
4. The user is then prompted to enter another query. This process continues until the user enters `exit`.

![](media/notebook_search.png)

You will be prompted to enter a query. Enter a query and press enter. The application will return a product list relevant to the query.

Here are some queries to try out:

- What is azure devops?
- How to use Azure Blueprints?
- Getting started with Azure Synapse and HDInsight

In [ ]:
# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    results = search(query, 5)
    display_results(results, query)